In [1]:
using Distributed

using Pkg


@everywhere begin 
    using ParallelTemperingMonteCarlo
    using DelimitedFiles
    using BenchmarkTools
end

In [2]:
using Random
@everywhere import ..MCRun: initialise_histograms!,updatehistogram!,update_max_stepsize!,sampling_step!,save_results,save_states
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 50.
tf = 160.
n_traj = 16

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details
mc_cycles = 1000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust, eq_percentage = 1)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
ico_13  = [[-0.0000000049,       -0.0000000044,       -0.0000000033],
[-0.0000007312,       -0.0000000014,        0.6554619119],
 [0.1811648930,      -0.5575692094,        0.2931316798],
[-0.4742970242,       -0.3445967289,        0.2931309525],
[-0.4742970303,        0.3445967144,        0.2931309494],
 [0.1811648830,        0.5575692066,        0.2931316748],
 [0.5862626299,        0.0000000022,        0.2931321262],
[-0.1811648928,       -0.5575692153,       -0.2931316813],
[-0.5862626397,       -0.0000000109,       -0.2931321327],
[-0.1811649028,        0.5575692007,       -0.2931316863],
 [0.4742970144,        0.3445967202,       -0.2931309590],
 [0.4742970205,       -0.3445967231,       -0.2931309559],
 [0.0000007214,       -0.0000000073,       -0.6554619185]]

#convert to Bohr
nmtobohr = 18.8973
copperconstant = 0.36258*nmtobohr
pos_cu13 = copperconstant*ico_13*1.5
AtoBohr = 1.8897259886


#boundary conditions 
bc_cu13 = SphericalBC(radius=12.0*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_cu13, bc_cu13)

#histogram information
n_bin = 100




100

In [3]:
function parallel_equilibration(mc_states,move_strat,mc_params,pot,ensemble,results)
    parallel_states = []

     #initialise state and potentials

    n_threads = Threads.nthreads()
    sample_index = Int64(floor(mc_params.eq_cycles / n_threads)) #number of eq cycles per thread

    a = atom_move_frequency(move_strat)
    v = vol_move_frequency(move_strat)
    r = rot_move_frequency(move_strat)# function init_parallel_RuNNer(pot::AbstractMLPotential; n_threads = Threads.nthreads())
    n_steps = a + v + r
    println()

    ebounds = [100. , -100.]

    for i_thread = 1:n_threads
        for i_eq = 1:sample_index
            
            i = i_thread*sample_index + i_eq

            mc_states = mc_cycle!(mc_states, move_strat, mc_params, pot, ensemble, n_steps, a, v, r)#mc cycle
            
            for i_traj = 1:mc_params.n_traj#check energy bounds
                if mc_states[i_traj].en_tot < ebounds[1]
                    ebounds[1] = mc_states[i_traj].en_tot
                end

                if mc_states[i_traj].en_tot > ebounds[2]
                    ebounds[2] = mc_states[i_traj].en_tot
                end

            end

            if rem(i, mc_params.n_adjust) == 0 #adjust stepsize
                for i_traj = 1:mc_params.n_traj
                    update_max_stepsize!(mc_states[i_traj], mc_params.n_adjust, a, v, r)
                end 
            end

        end
        
        states_vec = [MCState(mc_states[i_traj].temp,mc_states[i_traj].beta,mc_states[i_traj].config,pot_vector[i_thread];max_displ = mc_states[i_traj].max_displ ) for i_traj in 1:mc_params.n_traj] #initialise a new mc_states vector based on current state

        for i_traj = 1:mc_params.n_traj
            push!(states_vec[i_traj].ham, 0)
            push!(states_vec[i_traj].ham, 0)
        end

        push!(parallel_states,states_vec) #add to vector of parallel states


    end
    
    

    delta_en = initialise_histograms!(mc_params,mc_states,results, full_ham=false,e_bounds=ebounds) #start histogram

    println("equilibration done")
    
    return parallel_states,a,v,r,delta_en,n_threads

end

parallel_equilibration (generic function with 1 method)

In [4]:
import .MCRun.mc_cycle!
function mc_cycle!(mc_states, move_strat, mc_params, pot::ParallelMLPotential, ensemble, n_steps, a, v, r)
    file = RuNNer.writeinit(pot.dir;input_idx=pot.index)
    #this for loop creates n_traj perturbed atoms
    indices = []
    trials = []
    #we require parallelisation here, but will need to avoid a race condition
    for mc_state in mc_states
        #for i_step = 1:n_steps
            ran = rand(1:(a+v+r))
            trial_pos = atom_displacement(mc_state.config.pos[ran], mc_state.max_displ[1], mc_state.config.bc)
            writeconfig(file,mc_state.config,ran,trial_pos, pot.atomtype)
            push!(indices,ran)
            push!(trials,trial_pos)
        #end
    end
    #after which we require energy evaluations of the n_traj new configurations
    close(file)    
    energyvec = getRuNNerenergy(pot.dir,mc_params.n_traj; input_idx=pot.index)    
    #this replaces the atom_move! function
    #parallelisation here is fine
    for i in 1:mc_params.n_traj
        if metropolis_condition(ensemble, (mc_states[i].en_tot - energyvec[i]), mc_states[i].beta ) >=rand()
            mc_states[i].config.pos[indices[i]] = trials[i]
            mc_states[i].en_tot = energyvec[i]
            mc_states[i].count_atom[1] +=1
            mc_states[i].count_atom[2] += 1
        end
    end


    if rand() < 0.1 #attempt to exchange trajectories
        n_exc = rand(1:mc_params.n_traj-1)
        mc_states[n_exc].count_exc[1] += 1
        mc_states[n_exc+1].count_exc[1] += 1
        exc_acc = exc_acceptance(mc_states[n_exc].beta, mc_states[n_exc+1].beta, mc_states[n_exc].en_tot,  mc_states[n_exc+1].en_tot)
        if exc_acc > rand()
            mc_states[n_exc].count_exc[2] += 1
            mc_states[n_exc+1].count_exc[2] += 1
            mc_states[n_exc], mc_states[n_exc+1] = exc_trajectories!(mc_states[n_exc], mc_states[n_exc+1])
        end
    end


    return mc_states
end

mc_cycle! (generic function with 3 methods)

In [5]:
@everywhere function threadexchange(parallel_states,n_threads,idx)
    if rand() < 0.2  #20% change per trajectory of an attempted exchange
        thrid = rand(1:n_threads,2)
        if thrid[1] == thrid[2] && thrid[2] == n_threads
            thrid[2] = rand(1:n_threads-1)
        elseif thrid[1] == thrid[2] && thrid[2] != n_threads
            thrid[2] +=1
        end #which threads are talking

        exc_acc = exc_acceptance(parallel_states[thrid[1]][idx].beta,parallel_states[thrid[2]][idx].beta,parallel_states[thrid[1]][idx].en_tot,parallel_states[thrid[2]][idx].en_tot) #calc acceptance
        
        if exc_acc > rand() #if exchange is likely
            exc_trajectories!(parallel_states[thrid[1]][idx] ,parallel_states[thrid[2]][idx] )#swap
        end
     end
    
end



In [6]:
 runnerdir = "/home/ghun245/RuNNer-testing/Brass_potential/"
 atomtype="Cu"
# pot = AbstractMLPotential(runnerdir,atomtype)
# RuNNer.getRuNNerenergy(runnerdir,1;input_idx=2)

"Cu"

In [7]:
pot_vector = []

for i =1:Threads.nthreads()
    potential=ParallelMLPotential(runnerdir,"Cu",i)
    push!(pot_vector,potential)
end

mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot_vector[1] ; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]
    

results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)

In [10]:
parallel_states,a,v,r,delta_en,n_threads = parallel_equilibration(mc_states,move_strat,mc_params,pot_vector[1],ensemble,results)

(Any[MCState{Float64, 13, SphericalBC{Float64}}[MCState{Float64, 13, SphericalBC{Float64}}(50.0, 6315.499705261945, Config{13, SphericalBC{Float64}, Float64}(StaticArraysCore.SVector{3, Float64}[[-0.01357560085062383, 0.07397845641315581, 0.13078321516653835], [0.13383528664474098, 0.003456137642791279, 7.04811993740721], [2.0438414640958347, -5.761286590226343, 3.2999302311384087], [-4.874670455235371, -3.541653030973376, 3.01270453061964], [-4.931236512074463, 3.8020938382436644, 3.1268590818811615], [1.8619537075439925, 5.730514845094081, 3.0127119541839686], [6.396597241406714, -0.1882387669133309, 3.2082674440179235], [-1.868257199897207, -6.026474706790241, -2.9984610601378052], [-6.2369734766570035, 0.011829630881453501, -3.1295682022189686], [-1.8381775533379427, 5.878447126415563, -2.9969942939003085], [5.111958369239842, 3.5134261562294773, -3.1413048486750874], [4.977810402656452, -3.6425922295152917, -3.0897871786350253], [-0.146681160069665, 0.011217527401057414, -6.972679

In [11]:
function testparallelrunner(parallel_states,a,v,r,delta_en,n_threads,pot_vec)
    Threads.@threads for i=1:4
        for j=1:1000
            mc_cycle!(parallel_states[i], move_strat, mc_params, pot_vec[i], ensemble, 1, a, v, r)
        end
    end
end

function testrunner(parallel_states,a,v,r,delta_en,n_threads,pot_vec)
    for i=1:4
        for j=1:1000
            mc_cycle!(parallel_states[i], move_strat, mc_params, pot_vec[i], ensemble, 1, a, v, r)
        end
    end
end

testrunner (generic function with 1 method)

In [12]:
@benchmark testrunner(parallel_states,a,v,r,delta_en,n_threads,pot_vector)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 240.002 s (0.33% GC) to evaluate,
 with a memory estimate of 4.16 GiB, over 86794324 allocations.

In [13]:
@benchmark testparallelrunner(parallel_states,a,v,r,delta_en,n_threads,pot_vector)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 73.532 s (0.60% GC) to evaluate,
 with a memory estimate of 3.95 GiB, over 75731738 allocations.

In [14]:
# function testrun(input_idx)
#     run(`./RuNNer$input_idx.x $input_idx`)
# end
# #Threads.@threads
# for j=1:200 
#     Threads.@threads for i=1:8
#         testrun(i)    
#     end
# end
